In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/arc-prize-2024/arc-agi_training_solutions.json
/kaggle/input/arc-prize-2024/arc-agi_evaluation_solutions.json
/kaggle/input/arc-prize-2024/arc-agi_evaluation_challenges.json
/kaggle/input/arc-prize-2024/sample_submission.json
/kaggle/input/arc-prize-2024/arc-agi_training_challenges.json
/kaggle/input/arc-prize-2024/arc-agi_test_challenges.json


In [2]:
import json

def load_json(filepath):
    with open(filepath, 'r') as file:
        return json.load(file)

#load the data
training_solutions = load_json('/kaggle/input/arc-prize-2024/arc-agi_training_solutions.json')
evaluation_solutions = load_json('/kaggle/input/arc-prize-2024/arc-agi_evaluation_solutions.json')
training_challenges = load_json('/kaggle/input/arc-prize-2024/arc-agi_training_challenges.json')
evaluation_challenges = load_json('/kaggle/input/arc-prize-2024/arc-agi_evaluation_challenges.json')
test_challenges = load_json('/kaggle/input/arc-prize-2024/arc-agi_test_challenges.json')

In [3]:
def inspect_data(data, num_samples=1):
    for key, value in list(data.items())[:num_samples]:
        print(f"Key: {key}")
        print(f"Sample Data: {value}\n")
        
print("Training Solutions:")
inspect_data(training_solutions)

print("Training Challenges:")
inspect_data(training_challenges)

print("Evaluation Solutions:")
inspect_data(evaluation_solutions)

print("Evaluation Challenges:")
inspect_data(evaluation_challenges)

print("Test Challenges:")
inspect_data(test_challenges)

Training Solutions:
Key: 007bbfb7
Sample Data: [[[7, 0, 7, 0, 0, 0, 7, 0, 7], [7, 0, 7, 0, 0, 0, 7, 0, 7], [7, 7, 0, 0, 0, 0, 7, 7, 0], [7, 0, 7, 0, 0, 0, 7, 0, 7], [7, 0, 7, 0, 0, 0, 7, 0, 7], [7, 7, 0, 0, 0, 0, 7, 7, 0], [7, 0, 7, 7, 0, 7, 0, 0, 0], [7, 0, 7, 7, 0, 7, 0, 0, 0], [7, 7, 0, 7, 7, 0, 0, 0, 0]]]

Training Challenges:
Key: 007bbfb7
Sample Data: {'test': [{'input': [[7, 0, 7], [7, 0, 7], [7, 7, 0]]}], 'train': [{'input': [[0, 7, 7], [7, 7, 7], [0, 7, 7]], 'output': [[0, 0, 0, 0, 7, 7, 0, 7, 7], [0, 0, 0, 7, 7, 7, 7, 7, 7], [0, 0, 0, 0, 7, 7, 0, 7, 7], [0, 7, 7, 0, 7, 7, 0, 7, 7], [7, 7, 7, 7, 7, 7, 7, 7, 7], [0, 7, 7, 0, 7, 7, 0, 7, 7], [0, 0, 0, 0, 7, 7, 0, 7, 7], [0, 0, 0, 7, 7, 7, 7, 7, 7], [0, 0, 0, 0, 7, 7, 0, 7, 7]]}, {'input': [[4, 0, 4], [0, 0, 0], [0, 4, 0]], 'output': [[4, 0, 4, 0, 0, 0, 4, 0, 4], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 4, 0, 0, 0, 0, 0, 4, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 4, 0, 4, 0, 0,

In [4]:
#load training data
with open('/kaggle/input/arc-prize-2024/arc-agi_training_challenges.json') as f:
    training_challenges = json.load(f)
with open('/kaggle/input/arc-prize-2024/arc-agi_training_solutions.json') as f:
    training_solutions = json.load(f)

def prepare_training_data_padded(challenges, solutions):
    #calculate the maximum height and width for padding
    max_height = max(
        max(len(example['input']) for example in challenge['train'])
        for challenge in challenges.values()
    )
    max_width = max(
        max(len(row) for example in challenge['train'] for row in example['input'])
        for challenge in challenges.values()
    )
    
    print(f"Maximum height: {max_height}")
    print(f"Maximum width: {max_width}")
    
    def pad_grid(grid, target_height, target_width):
        padded_grid = np.zeros((target_height, target_width), dtype=int)
        for i, row in enumerate(grid):
            padded_grid[i, :len(row)] = row
        return padded_grid

    X_train = []
    y_train = []

    for key in challenges.keys():
        challenge = challenges[key]
        solution = solutions[key]
        
        #add training data
        for example in challenge['train']:
            input_grid = example['input']
            output_grid = example['output']
            padded_input = pad_grid(input_grid, max_height, max_width)
            padded_output = pad_grid(output_grid, max_height, max_width)
            X_train.append(padded_input)
            y_train.append(padded_output)
    
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    
    print(f"X_train shape: {X_train.shape}")
    print(f"y_train shape: {y_train.shape}")
    
    #save the arrays
    np.save('X_train.npy', X_train)
    np.save('y_train.npy', y_train)
    
    return X_train, y_train

X_train, y_train = prepare_training_data_padded(training_challenges, training_solutions)

X_train_loaded = np.load('X_train.npy')
y_train_loaded = np.load('y_train.npy')

print(f"Loaded X_train shape: {X_train_loaded.shape}")
print(f"Loaded y_train shape: {y_train_loaded.shape}")

Maximum height: 30
Maximum width: 30
X_train shape: (1302, 30, 30)
y_train shape: (1302, 30, 30)
Loaded X_train shape: (1302, 30, 30)
Loaded y_train shape: (1302, 30, 30)


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TransformerModel(nn.Module):
    """A neural network that processes sequences of data using attention mechanisms to understand context and relationships."""

    def __init__(self, input_dim, model_dim, num_heads, num_layers, output_dim):
        super(TransformerModel, self).__init__()
        
        #embedding layer for input sequences
        self.embedding = nn.Embedding(input_dim, model_dim)
        
        #Transformer module
        self.transformer = nn.Transformer(
            d_model=model_dim, 
            nhead=num_heads, 
            num_encoder_layers=num_layers, 
            num_decoder_layers=num_layers,
            batch_first=True  #set batch_first to True
        )
        
        #output linear layer to transform the transformer output to desired output dimensions
        self.fc_out = nn.Linear(model_dim, output_dim)

    def forward(self, src, tgt):
        """
        Forward pass through the model.
        
        Parameters:
        src (torch.Tensor): Source input tensor of shape (batch_size, seq_len, input_dim)
        tgt (torch.Tensor): Target input tensor of shape (batch_size, seq_len, input_dim)
        
        Returns:
        torch.Tensor: Output tensor of shape (batch_size, seq_len, output_dim)
        """
        #apply embeddings to the source and target sequences
        src_embedded = self.embedding(src) 
        tgt_embedded = self.embedding(tgt)
        
        #pass through the Transformer
        transformer_output = self.transformer(src_embedded, tgt_embedded)
        
        #pass through the final linear layer
        output = self.fc_out(transformer_output)
        
        return output

In [6]:
class GNNModel(nn.Module):
    """A neural network designed for graph data that processes nodes and edges to learn relationships in graphs."""

    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GNNModel, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        
        #define layers
        self.conv1 = nn.Linear(input_dim, hidden_dim)  #first graph convolution layer
        self.conv2 = nn.Linear(hidden_dim, hidden_dim)  #second graph convolution layer
        self.fc_out = nn.Linear(hidden_dim, output_dim)  #output layer

    def forward(self, graph_data):
        """
        Forward pass through the model.

        Parameters:
        graph_data (dict): A dictionary containing 'nodes' and 'edges'.
                           - 'nodes' (torch.Tensor): Node features of shape (num_nodes, input_dim)
                           - 'edges' (torch.Tensor): Edge list of shape (num_edges, 2), indicating connections between nodes

        Returns:
        torch.Tensor: Output tensor with node-level features transformed by the model
        """
        nodes = graph_data['nodes']
        edges = graph_data['edges'] 

        #pass node features through the first graph convolution layer
        x = F.relu(self.conv1(nodes))
        
        #pass through the second graph convolution layer
        x = F.relu(self.conv2(x))
        
        #mean pooling
        x = x.mean(dim=0, keepdim=True)

        #pass through the final linear layer
        x = self.fc_out(x)

        return x

In [7]:
class Rule:
    """A class that defines a symbolic rule for transforming an input grid."""
    
    def __init__(self, rule_func):
        """
        Initialize a rule with a transformation function.

        Parameters:
        rule_func (function): A function that takes output_grid and input_grid, and returns a transformed output_grid.
        """
        self.rule_func = rule_func

    def apply(self, output_grid, input_grid):
        """
        Apply the rule to transform the input grid.

        Parameters:
        output_grid (numpy.ndarray): The current output grid to be transformed.
        input_grid (numpy.ndarray): The input grid that will be used for transformation.

        Returns:
        numpy.ndarray: The transformed output grid after applying the rule.
        """
        return self.rule_func(output_grid, input_grid)


class SymbolicRuleModel:
    """A model that applies predefined rules to process input data and generate outputs based on symbolic logic."""

    def __init__(self, rules):
        """
        Initialize the symbolic rule model with a list of rules.

        Parameters:
        rules (list of Rule): A list of Rule objects to be applied in sequence.
        """
        self.rules = rules  #a list of Rule objects

    def apply_rules(self, input_grid):
        """
        Applies predefined rules to the input grid.

        Parameters:
        input_grid (numpy.ndarray): The input grid to which rules will be applied.

        Returns:
        numpy.ndarray: The output grid after applying all the rules.
        """
        output_grid = np.zeros_like(input_grid)  #initialize output grid with zeros
        for rule in self.rules:
            output_grid = rule.apply(output_grid, input_grid)
        return output_grid

In [8]:
class EvolutionaryOptimizer:
    """An optimizer that adjusts model parameters using evolutionary algorithms to refine performance."""

    def __init__(self, model, population_size, generations, mutation_rate):
        """
        Initialize the evolutionary optimizer.

        Parameters:
        model (torch.nn.Module): The model to optimize.
        population_size (int): Number of solutions in the population.
        generations (int): Number of generations to evolve.
        mutation_rate (float): Probability of mutation for each parameter.
        """
        self.model = model
        self.population_size = population_size
        self.generations = generations
        self.mutation_rate = mutation_rate

    def initialize_population(self):
        """Initialize the population with random model parameters."""
        population = []
        for _ in range(self.population_size):
            params = [torch.randn_like(p) for p in self.model.parameters()]
            population.append(params)
        return population

    def evaluate_fitness(self, population, X_train, y_train):
        """Evaluate the fitness of each solution in the population."""
        fitness_scores = []
        for params in population:
            self.set_params(params)
            outputs = self.model(X_train, X_train, {})  #use X_train as dummy input
            loss = F.mse_loss(outputs, y_train)
            fitness_scores.append(-loss.item())  #negate loss to maximize fitness
        return fitness_scores

    def set_params(self, params):
        """Set the model parameters."""
        with torch.no_grad():
            for param, new_param in zip(self.model.parameters(), params):
                param.copy_(new_param)

    def select_parents(self, population, fitness_scores):
        """Select parents based on fitness scores."""
        indices = np.argsort(fitness_scores)[-self.population_size // 2:]
        parents = [population[i] for i in indices]
        return parents

    def crossover(self, parents):
        """Create offspring by combining parents."""
        offspring = []
        for _ in range(self.population_size - len(parents)):
            p1, p2 = np.random.choice(parents, 2, replace=False)
            child = [torch.where(torch.rand_like(p1[0]) < 0.5, p1[i], p2[i]) for i in range(len(p1))]
            offspring.append(child)
        return offspring

    def mutate(self, population):
        """Apply mutation to the population."""
        for i in range(len(population)):
            if np.random.rand() < self.mutation_rate:
                for j in range(len(population[i])):
                    if np.random.rand() < self.mutation_rate:
                        mutation = torch.randn_like(population[i][j]) * 0.1  #small mutation
                        population[i][j] += mutation

    def optimize(self, X_train, y_train):
        """Run the evolutionary optimization algorithm."""
        population = self.initialize_population()
        for generation in range(self.generations):
            fitness_scores = self.evaluate_fitness(population, X_train, y_train)
            parents = self.select_parents(population, fitness_scores)
            offspring = self.crossover(parents)
            self.mutate(parents + offspring)
            population = parents + offspring
            
            best_fitness = max(fitness_scores)
            print(f"Generation {generation + 1}: Best Fitness = {best_fitness}")

        #return the best solution from the last generation
        best_index = np.argmax(fitness_scores)
        best_params = population[best_index]
        self.set_params(best_params)

In [9]:
class HybridModel(nn.Module):
    """A model integrating Transformer, GNN, and Neuro-Symbolic methods for diverse data processing and reasoning."""

    def __init__(self, transformer_model, gnn_model, neuro_symbolic_model):
        super(HybridModel, self).__init__()
        self.transformer = transformer_model
        self.gnn = gnn_model
        self.neuro_symbolic = neuro_symbolic_model

    def forward(self, src, tgt, graph_data):
        transformer_output = self.transformer(src, tgt)  #Transformer output
        gnn_output = self.gnn(graph_data)  #GNN output

        #apply symbolic rules to the transformer output
        transformer_output_np = transformer_output.detach().cpu().numpy()  #convert to numpy for rule application
        neuro_symbolic_output_np = self.neuro_symbolic.apply_rules(transformer_output_np)
        neuro_symbolic_output = torch.tensor(neuro_symbolic_output_np, dtype=torch.float32, device=transformer_output.device)

        return transformer_output, gnn_output, neuro_symbolic_output

In [10]:
#Yambi (which means "Welcome" in kikongo) Model
class Yambi(nn.Module):
    """A model that combines TransformerModel, GNNModel, and NeuroSymbolicModel to process and integrate different types of data."""

    def __init__(self, transformer_model, gnn_model, neuro_symbolic_model):
        super(Yambi, self).__init__()
        self.hybrid_model = HybridModel(transformer_model, gnn_model, neuro_symbolic_model)
        self.optimizer = EvolutionaryOptimizer(self, population_size=10, generations=5, mutation_rate=0.01)

    def forward(self, src, tgt, graph_data):
        """Process input data through the hybrid model."""
        return self.hybrid_model(src, tgt, graph_data)

    def optimize(self, X_train, y_train):
        """Optimize model parameters using evolutionary algorithms."""
        self.optimizer.optimize(X_train, y_train)

In [11]:
#define Symbolic Rules
def rule_example1(output_grid, input_grid):
    """An example rule that fills the output grid with the mean value of the input grid."""
    mean_value = np.mean(input_grid)
    output_grid[:] = mean_value
    return output_grid

def rule_example2(output_grid, input_grid):
    """An example rule that sets the output grid to be the square of the input grid."""
    output_grid[:] = np.square(input_grid)
    return output_grid

rules = [
    Rule(rule_func=rule_example1),
    Rule(rule_func=rule_example2)
]

In [12]:
#instantiate SymbolicRuleModel with Rules
neuro_symbolic_model = SymbolicRuleModel(rules=rules)

#instantiate the Yambi model
transformer_model = TransformerModel(input_dim=100, model_dim=64, num_heads=8, num_layers=4, output_dim=30)
gnn_model = GNNModel(input_dim=100, hidden_dim=64, output_dim=30)

yambi_model = Yambi(transformer_model, gnn_model, neuro_symbolic_model)

In [13]:
from torch.utils.data import DataLoader, TensorDataset

#convert the data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_loaded, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_loaded, dtype=torch.float32)

#create a TensorDataset and DataLoader
batch_size = 32
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [14]:
import torch.optim as optim

#define the loss function and optimizer
criterion = nn.MSELoss() 
optimizer = optim.Adam(yambi_model.parameters(), lr=0.001)